In [ ]:
from google.colab import drive
import os
import zipfile
import shutil
drive.mount('/content/drive')
zip_path = "/content/drive/My Drive/dataset.zip"
extract_path = "/content/dataset"
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import os
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input

base_model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')
feature_extractor = tf.keras.Model(inputs=base_model.input, outputs=base_model.output)

def extract_features(folder):
    X, y = [], []
    labels = sorted(os.listdir(folder))
    label_dict = {label: idx for idx, label in enumerate(labels)}

    for label in labels:
        label_path = os.path.join(folder, label)
        if not os.path.isdir(label_path):
            continue

        for img_name in os.listdir(label_path):
            img_path = os.path.join(label_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            if img is None:
                continue

            img = cv2.resize(img, (224, 224))
            img = preprocess_input(img)
            img = np.expand_dims(img, axis=0)
            feature = feature_extractor.predict(img)
            X.append(feature.flatten())
            y.append(label_dict[label])

    return np.array(X), np.array(y), label_dict
train_dir = "/content/dataset/dataset/train"
test_dir = "/content/dataset/dataset/test"
X_train, y_train, label_dict = extract_features(train_dir)
X_test, y_test, _ = extract_features(test_dir)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_accuracies = []
cv_precisions = []
cv_recalls = []
cv_f1_scores = []

for train_idx, val_idx in kfold.split(X_train, y_train):
    X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
    y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

    svm = SVC(kernel="rbf", C=10, gamma="scale")
    svm.fit(X_train_fold, y_train_fold)

    y_pred_fold = svm.predict(X_val_fold)

    report = classification_report(y_val_fold, y_pred_fold, output_dict=True)

    cv_precisions.append(report["weighted avg"]["precision"])
    cv_recalls.append(report["weighted avg"]["recall"])
    cv_f1_scores.append(report["weighted avg"]["f1-score"])

    fold_accuracy = report["accuracy"]
    cv_accuracies.append(fold_accuracy)

    print(f"Classification report for fold:\n{classification_report(y_val_fold, y_pred_fold)}")

avg_accuracy = np.mean(cv_accuracies)
avg_precision = np.mean(cv_precisions)
avg_recall = np.mean(cv_recalls)
avg_f1 = np.mean(cv_f1_scores)

print(f"\n===== Average Cross-Validation Results =====")
print(f"Accuracy: {avg_accuracy:.4f}")
print(f"Precision: {avg_precision:.4f}")
print(f"Recall: {avg_recall:.4f}")
print(f"F1-score: {avg_f1:.4f}")


In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import os
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input

base_model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')
feature_extractor = tf.keras.Model(inputs=base_model.input, outputs=base_model.output)

def extract_features(folder):
    X, y = [], []
    labels = sorted(os.listdir(folder))
    label_dict = {label: idx for idx, label in enumerate(labels)}

    for label in labels:
        label_path = os.path.join(folder, label)
        if not os.path.isdir(label_path):
            continue

        for img_name in os.listdir(label_path):
            img_path = os.path.join(label_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            if img is None:
                continue

            img = cv2.resize(img, (224, 224))
            img = preprocess_input(img)
            img = np.expand_dims(img, axis=0)
            feature = feature_extractor.predict(img)
            X.append(feature.flatten())
            y.append(label_dict[label])

    return np.array(X), np.array(y), label_dict

train_dir = "/content/dataset/dataset/train"
test_dir = "/content/dataset/dataset/test"
X_train, y_train, label_dict = extract_features(train_dir)
X_test, y_test, _ = extract_features(test_dir)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

svm = SVC(kernel="rbf", C=10, gamma="scale")
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)

report = classification_report(y_test, y_pred, output_dict=True)

avg_precision = report["weighted avg"]["precision"]
avg_recall = report["weighted avg"]["recall"]
avg_f1 = report["weighted avg"]["f1-score"]
accuracy = report["accuracy"]

print(f"Classification report:\n{classification_report(y_test, y_pred)}")

print(f"\n===== Final Results =====")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {avg_precision:.4f}")
print(f"Recall: {avg_recall:.4f}")
print(f"F1-score: {avg_f1:.4f}")


Streaming output truncated to the last 5000 lines.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/s

In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import os
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

# 使用 ResNet50 作為特徵提取器
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
feature_extractor = tf.keras.Model(inputs=base_model.input, outputs=base_model.output)

def extract_features(folder):
    X, y = [], []
    labels = sorted(os.listdir(folder))
    label_dict = {label: idx for idx, label in enumerate(labels)}

    for label in labels:
        label_path = os.path.join(folder, label)
        if not os.path.isdir(label_path):
            continue

        for img_name in os.listdir(label_path):
            img_path = os.path.join(label_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            if img is None:
                continue

            img = cv2.resize(img, (224, 224))
            img = preprocess_input(img)
            img = np.expand_dims(img, axis=0)
            feature = feature_extractor.predict(img)
            X.append(feature.flatten())
            y.append(label_dict[label])

    return np.array(X), np.array(y), label_dict

train_dir = "/content/dataset/dataset/train"
test_dir = "/content/dataset/dataset/test"
X_train, y_train, label_dict = extract_features(train_dir)
X_test, y_test, _ = extract_features(test_dir)

# 特徵標準化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 訓練 SVM
svm = SVC(kernel="rbf", C=10, gamma="scale")
svm.fit(X_train, y_train)

# 預測
y_pred = svm.predict(X_test)

# 生成分類報告 (轉為字典格式)
report = classification_report(y_test, y_pred, output_dict=True)

# 計算並存儲 precision, recall, f1-score
avg_precision = report["weighted avg"]["precision"]
avg_recall = report["weighted avg"]["recall"]
avg_f1 = report["weighted avg"]["f1-score"]
accuracy = report["accuracy"]

print(f"Classification report:\n{classification_report(y_test, y_pred)}")

print(f"\n===== Final Results =====")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {avg_precision:.4f}")
print(f"Recall: {avg_recall:.4f}")
print(f"F1-score: {avg_f1:.4f}")


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s

In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import os
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input

# 初始化 EfficientNetB0 作為特徵提取器
base_model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')
feature_extractor = tf.keras.Model(inputs=base_model.input, outputs=base_model.output)

def extract_features(folder):
    X, y = [], []
    labels = sorted(os.listdir(folder))
    label_dict = {label: idx for idx, label in enumerate(labels)}

    for label in labels:
        label_path = os.path.join(folder, label)
        if not os.path.isdir(label_path):
            continue

        for img_name in os.listdir(label_path):
            img_path = os.path.join(label_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            if img is None:
                continue

            img = cv2.resize(img, (224, 224))
            img = preprocess_input(img)
            img = np.expand_dims(img, axis=0)
            feature = feature_extractor.predict(img)
            X.append(feature.flatten())
            y.append(label_dict[label])

    return np.array(X), np.array(y), label_dict

train_dir = "/content/dataset/dataset/train"
test_dir = "/content/dataset/dataset/test"
X_train, y_train, label_dict = extract_features(train_dir)
X_test, y_test, _ = extract_features(test_dir)

# 特徵標準化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 訓練 SVM
svm = SVC(kernel="rbf", C=10, gamma="scale")
svm.fit(X_train, y_train)

# 預測
y_pred = svm.predict(X_test)

# 生成分類報告 (轉為字典格式)
report = classification_report(y_test, y_pred, output_dict=True)

# 計算並存儲 precision, recall, f1-score
avg_precision = report["weighted avg"]["precision"]
avg_recall = report["weighted avg"]["recall"]
avg_f1 = report["weighted avg"]["f1-score"]
accuracy = report["accuracy"]

print(f"Classification report:\n{classification_report(y_test, y_pred)}")

print(f"\n===== Final Results =====")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {avg_precision:.4f}")
print(f"Recall: {avg_recall:.4f}")
print(f"F1-score: {avg_f1:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━